<a href="https://colab.research.google.com/github/varunkr24/Machine-Learning/blob/Python/Portfolio_Call_Routing_DTC_Clustering_EA_Propensity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from datetime import date
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn import preprocessing
import pyodbc
import pickle
import math
import time
import warnings
import os
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")
sys.setrecursionlimit(10000)

In [ ]:
#Loading the data from model table in SQL Server

sql_conn = pyodbc.connect("DSN=CNPPEDW05")

sql = "select * from Analytics_RPT.dbo.DTC_SBR_PortfolioLoanSegmentData"

df = pd.read_sql(sql, sql_conn)

df.shape

In [ ]:
df.head()

In [ ]:
df['AsOfDt'].unique()

### Clustering

In [ ]:
object_dict = {'OrigLoanRefiPurpose': ['R&T',
  'Purchase','PURCHASE',
  'CASHOUT - OTHER',
  'CASHOUT - HOME IMPROV',
  'CASHOUT - DEBT CONSOL',
  'N/A'],
 'prop_NumberOfUnits' : ['ONE','TWO','THREE','FOUR','UNKNOWN'],              
 'OrigLoanType': ['CONVENTIONAL',
  'FHA',
  'VA',
  'USDA',
  'Bond',
  'HELOC',
  'USDA/RURAL HOUSING',
  'PIH'],
 'PropState': ['TX',
  'VA',
  'MO',
  'NM',
  'CO',
  'CA',
  'MN',
  'UT',
  'DC',
  'IN',
  'FL',
  'KS',
  'NJ',
  'NY',
  'WA',
  'PA',
  'IL',
  'NC',
  'IA',
  'MI',
  'AZ',
  'TN',
  'GA',
  'NV',
  'OR',
  'NH',
  'SC',
  'MD',
  'OK',
  'NE',
  'KY',
  'LA',
  'WI',
  'ME',
  'AR',
  'WY',
  'SD',
  'AL',
  'MA',
  'OH',
  'WV',
  'ND',
  'MT',
  'ID',
  'RI',
  'CT',
  'HI',
  'DE',
  'MS',
  'VT',
  'AK',
  'VI',
  'PR',
  'GU'],
 'OrigLoanPurpose': ['REFINANCE', 'PURCHASE','UNKNOWN'],
 'orig_years': ['< 1','1-3 YRS','3-5 YRS','5+'],
 'cd_bill_meth':['0','1','2','4','5'],
 'BenefitFlag_Prev_90_days':['0','1'],
 'BenefitCategory_Prev_90_days':['NO INFO AVAILABLE', 'ELIMINATE MI', 'INVESTMENT/2ND HOME',
                                 'NO BENEFIT', 'CASHOUT', 'FHA STREAMLINE', 'RATE REDUCTION',
                                 'PAYMENT REDUCTION', 'REDUCE MI', 'VA IRRRL', 'CREDIT TRIGGER'],
 'OccType':['OWNER OCCUPIED', 'OCCUPIED (NAME UNKNOWN)', 'UNKNOWN',
            'INVESTMENT', 'VACANT', 'SECOND HOME']}

In [ ]:
final_cols =['term_orig','orig_years','HomePriceAppreciation', 
             'CurrentPIPmt', 'CurrRate', 'orig_LTV',
            'Val_waterfall', 'bal_orig',  'LandSquarefeet',
            'LandValue', 'CombinedMonthlyIncomeH2O', 'AccountAssetsH2O',
            'OrigLoanDTI', 'TotalNetWorth', 'HomeEquityAmt', 'UPB',
            'OrigLoanRefiPurpose', 'OrigLoanType', 'PropState',
            'OrigLoanPurpose','prop_NumberOfUnits','NumberOfBedrooms',
            'BenefitFlag_Prev_90_days','BenefitCategory_Prev_90_days',
            'cd_bill_meth','OccType']

In [ ]:
num_cols = ['term_orig',
 'HomePriceAppreciation',
 'CurrentPIPmt',
 'CurrRate',
 'orig_LTV',
 'Val_waterfall',
 'bal_orig',
 'LandSquarefeet',
 'LandValue',
 'CombinedMonthlyIncomeH2O',
 'AccountAssetsH2O',
 'OrigLoanDTI',
 'TotalNetWorth',
 'HomeEquityAmt',
 'UPB',
 'NumberOfBedrooms']
object_cols = ['OrigLoanRefiPurpose', 'OrigLoanType', 'PropState', 'OrigLoanPurpose','orig_years','prop_NumberOfUnits',
              'BenefitFlag_Prev_90_days','BenefitCategory_Prev_90_days','cd_bill_meth','OccType']

In [ ]:
prop_dict = {1:'one',2:'two',3:'three',4:'four',np.nan:'unknown'}
df['prop_NumberOfUnits'].apply(lambda x: prop_dict.get(x)).isnull().sum()

In [ ]:
class coalesce(BaseEstimator, TransformerMixin):
    
    def __init__(self,YearBuilt,Year_Built,prop_yr_blt,inf_NumberOfBathrooms,serv_NumberOfBathrooms,serv_NumberOfBedrooms,inf_NumberOfBedrooms,LandSquarefeet,Land_Squarefeet,prop_NumberOfUnits):
        self.YearBuilt = YearBuilt
        self.Year_Built = Year_Built
        self.prop_yr_blt = prop_yr_blt
        self.inf_NumberOfBathrooms = inf_NumberOfBathrooms
        self.serv_NumberOfBathrooms = serv_NumberOfBathrooms
        self.serv_NumberOfBedrooms = serv_NumberOfBedrooms
        self.inf_NumberOfBedrooms = inf_NumberOfBedrooms
        self.LandSquarefeet = LandSquarefeet
        self.Land_Squarefeet = Land_Squarefeet
        self.prop_NumberOfUnits = prop_NumberOfUnits
        self.prop_dict = {1:'one',2:'two',3:'three',4:'four'}
    
    def fit(self, X, y=None):
        return self

    
    def transform(self, X):
        X[self.prop_NumberOfUnits] = X[self.prop_NumberOfUnits].apply(lambda x: self.prop_dict.get(x))
        X.loc[X[self.YearBuilt].isnull(),self.YearBuilt] = X.loc[X[self.YearBuilt].isnull(),self.Year_Built]
        X.loc[X[self.prop_yr_blt].isnull(),self.prop_yr_blt] = X.loc[X[self.prop_yr_blt].isnull(),self.YearBuilt]
        X[self.inf_NumberOfBathrooms] = X[self.inf_NumberOfBathrooms].astype('float64')
        X[self.inf_NumberOfBathrooms] = X[self.inf_NumberOfBathrooms].apply(lambda x:x/100 if x%100 == 0 else x)
        X['NumberOfBathrooms'] = X[self.serv_NumberOfBathrooms]
        X.loc[X['NumberOfBathrooms'].isnull(),'NumberOfBathrooms'] = X.loc[X['NumberOfBathrooms'].isnull(),self.inf_NumberOfBathrooms]
        X['NumberOfBedrooms'] = X[self.serv_NumberOfBedrooms]
        X.loc[X['NumberOfBedrooms'].isnull(),'NumberOfBedrooms'] = X.loc[X['NumberOfBedrooms'].isnull(),self.inf_NumberOfBedrooms]
        X.loc[X[self.LandSquarefeet].isnull(),self.LandSquarefeet] = X.loc[X[self.LandSquarefeet].isnull(),self.Land_Squarefeet]
        X.drop([self.YearBuilt,self.Year_Built,self.serv_NumberOfBathrooms,self.inf_NumberOfBathrooms,self.serv_NumberOfBedrooms,self.inf_NumberOfBedrooms,self.Land_Squarefeet],axis=1,inplace=True)
        return X

In [ ]:
#calculate the PIPmt
# p = principal
# i = interest
# t = term
def PIpmt(p,i,t):
    i = i/1200
    n = int(t/30)
    return (p*i*pow(i+1,n))/(pow(i+1,n-1))

In [ ]:
def year_fun(x):
    if np.floor(x/365)< 1:
        return '< 1'
    elif np.floor(x/365) < 3:
        return '1-3 yrs'
    elif np.floor(x/365) < 6:
        return '3-5 yrs'
    else:
        return '5+'

In [ ]:
class feature_additions(BaseEstimator, TransformerMixin):
    
    def __init__(self,prevailing_rate_curr,prevailing_rate_orig,Val_waterfall,HomeEquityAmt,bal_orig,term_orig,AsOfDt,orig_dt,FundDt):
        self.prevailing_rate_curr = prevailing_rate_curr
        self.prevailing_rate_orig = prevailing_rate_orig
        self.Val_waterfall = Val_waterfall
        self.HomeEquityAmt = HomeEquityAmt
        self.bal_orig = bal_orig
        self.term_orig = term_orig
        self.AsOfDt = AsOfDt
        self.orig_dt = orig_dt
        self.FundDt = FundDt
        
    def fit(self, X, y=None):
        return self

    
    def transform(self, X):
        X['orig_years'] = ((pd.to_datetime(X[self.AsOfDt]) - pd.to_datetime(X[self.orig_dt])).dt.days).apply(year_fun)
        X['UPB'] = X[self.Val_waterfall]-X[self.HomeEquityAmt]
        X['diff_prevailing_rate'] = X[self.prevailing_rate_curr] - X[self.prevailing_rate_orig]
        X['CurrPIPmt'] = X.apply(lambda x: PIpmt(x['UPB'],x[self.prevailing_rate_curr],x[self.term_orig]),axis=1)
        X['pastPIpmt'] = X.apply(lambda x: PIpmt(x[self.bal_orig],x[self.prevailing_rate_orig],x[self.term_orig]),axis=1)
        X['diff_PIPmt'] = X['CurrPIPmt']-X['pastPIpmt']
        return X

In [ ]:
class ColumnTypeChange_Imputation(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_cols, cat_cols,final_cols):
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.final_cols = final_cols
    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.num_cols] = X[self.num_cols].astype('float64')#.fillna(-999)
        X[self.cat_cols] = X[self.cat_cols].fillna('Unknown').apply(lambda x: x.astype(str).str.upper()).astype('object')
        return X[['loan_num_srvcr','AsOfDt']+self.final_cols]

In [ ]:
class Missing(BaseEstimator, TransformerMixin):
    
    def __init__(self, total_cols = None):
        None
    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['missing'] = X.apply(lambda x: np.count_nonzero(x == 'UNKNOWN'), axis=1)
        X['missing'] = X['missing']+X.isnull().sum(axis=1)
        return X

In [ ]:
class Numerical_Outlier_Preprocess(BaseEstimator, TransformerMixin):
    
    def __init__(self, numerical_preprocess_cols):
        self.numerical_preprocess_cols = numerical_preprocess_cols
        
    def fit(self, X, y=None):
        
        self.outlier_dict = {}
        for col in self.numerical_preprocess_cols:
            p = X[col].quantile([0.05,0.95]).values
            self.outlier_dict[col] = list()            
            self.outlier_dict.get(col).append(p[0])
            self.outlier_dict.get(col).append(p[1])
           
        return self
    
    def transform(self, X):
        
        X = X.copy()
        for col in self.numerical_preprocess_cols:
            X.loc[X[col] <= self.outlier_dict.get(col)[0],col] = self.outlier_dict.get(col)[0]
            X.loc[X[col] >= self.outlier_dict.get(col)[1],col] = self.outlier_dict.get(col)[1]
        return X

In [ ]:
class Numerical_imputer(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_cols,object_cols,object_dict):
        self.num_cols = num_cols
        self.object_cols = object_cols
        self.object_dict = object_dict
        
    def fit(self, X, y=None):
        self.imputer_dict = {}
        self.mode_dict = {}
        for feature in self.num_cols:
            self.imputer_dict[feature] = X[feature].mean()
        for col in self.object_cols:
            self.mode_dict[col] = X[col].mode()[0]
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.num_cols:
            X[feature].fillna(self.imputer_dict.get(feature), inplace=True)
        for col in self.object_cols:
            X[col] = X[col].apply(lambda x: x if x in self.object_dict[col] else self.mode_dict[col])
        return X

In [ ]:
class label_encoding(BaseEstimator, TransformerMixin):
    
    def __init__(self, object_cols):
        self.object_cols = object_cols
        
    def fit(self, X, y=None):
        self.d = defaultdict(LabelEncoder)
        self.fit = X[self.object_cols].apply(lambda x: self.d[x.name].fit(x))
        
        return self

    def transform(self, X):
        X = X.copy()
        X[self.object_cols] = X[self.object_cols].apply(lambda x: self.d[x.name].transform(x))
        
        return X

In [ ]:
class standardize(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_cols):
        self.num_cols = num_cols
        
    def fit(self, X, y=None):
        self.preprocessor = preprocessing.MinMaxScaler().fit(X[num_cols])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.num_cols] = self.preprocessor.transform(X[self.num_cols])
    
        return X


#### Model

In [ ]:
cat_features = [1, 16, 17, 18, 19, 20, 22, 23, 24, 25]

In [ ]:
cat_features

In [ ]:
class kprototype_clustering(BaseEstimator, TransformerMixin):
    
    def __init__(self,cat_features):
        self.cat_features = cat_features
        
    def fit(self, X, y=None):
        self.total_cols = X.columns.to_list()
        self.total_cols.remove('loan_num_srvcr')
        self.total_cols.remove('AsOfDt')
        self.total_cols.remove('missing')
        kproto = KPrototypes(n_clusters=8, init='Huang',verbose=0,max_iter=10, n_init=10, n_jobs=-2,random_state=42) 
        X_full = X[self.total_cols]
        self.kproto_model = kproto.fit(np.array(X[self.total_cols]),categorical=self.cat_features)        
        return self

    def transform(self, X):
        X = X.copy()

        X_missing = X[X['missing'] >= 5]
        pred_missing = pd.DataFrame({'loan_num_srvcr': X_missing['loan_num_srvcr'],'AsOfDt':X_missing['AsOfDt'], 'cluster': 8})
        X_remaining = X[(X['missing'] < 5)] 
        
        if X_remaining.shape[0] >= 1:
            clusters = self.kproto_model.predict(np.array(X_remaining[self.total_cols]),categorical=self.cat_features)
            pred_remaining = pd.DataFrame({'loan_num_srvcr': X_remaining['loan_num_srvcr'],'AsOfDt':X_remaining['AsOfDt'],'cluster': clusters})
        else:
            pred_remaining = pd.DataFrame()
        
        total_pred_df = pd.concat([pred_missing, pred_remaining], ignore_index=True)

        total_pred_final_df = total_pred_df.drop_duplicates(keep='first')
        
        return total_pred_final_df


In [ ]:
filename = 'Port_call_routing_clustering_V2.pkl'
port_clustering_pipeline = pickle.load(open(filename, 'rb'))

In [ ]:
clustering_test_preds = port_clustering_pipeline.transform(df)
clustering_test_preds.shape

In [ ]:
clustering_test_preds.head()

**EA Cashout and Refiance Propensity model**

In [ ]:
#Loading the data from model table in SQL Server

sql_conn = pyodbc.connect("DSN=CNPPEDW05")

sql = "select * from analytics_dm.ds.mdScoresPropensityRefinance"

refi_propensity = pd.read_sql(sql, sql_conn)

refi_propensity.head()

In [ ]:
refi_propensity.shape

In [ ]:
refi_propensity['ScoringDt'].unique()

In [ ]:
refi_propensity.groupby(['ScoringDt','RefinanceModelId']).size()

In [ ]:
refi_propensity['RefinanceModelId'].unique()

In [ ]:
refi_propensity.groupby('RefinanceModelId').size()

In [ ]:
# 52 - RateTerm, 44 - CashOut 
propensity_rateterm_cashout = refi_propensity[refi_propensity['RefinanceModelId'].isin([52,44])]
propensity_rateterm_cashout.shape

In [ ]:
propensity_rateterm_cashout['ScoringDt'].max()

In [ ]:
propensity_rateterm_cashout.dtypes

In [ ]:
propensity_rateterm_cashout['ScoringDt'] = pd.to_datetime(propensity_rateterm_cashout['ScoringDt'], format='%Y-%m-%d')

In [ ]:
propensity_rateterm_cashout_latest = propensity_rateterm_cashout[propensity_rateterm_cashout['ScoringDt']==propensity_rateterm_cashout['ScoringDt'].max()]

In [ ]:
propensity_rateterm_cashout_latest.shape

In [ ]:
propensity_rateterm_cashout_latest['LoanNumber'].nunique()

In [ ]:
propensity_rateterm_cashout_latest.head()

In [ ]:
propensity_rateterm_cashout_latest['PropensityScore'].max()

In [ ]:
final_propensity = propensity_rateterm_cashout_latest.groupby('LoanNumber').agg({'PropensityScore':'sum'}).reset_index()
final_propensity.shape

In [ ]:
final_propensity.head()

In [ ]:
clustering_test_preds.head()

In [ ]:
df_final = pd.merge(clustering_test_preds,final_propensity,how='left',left_on='loan_num_srvcr', right_on = 'LoanNumber')
df_final.shape

In [ ]:
df_final.isnull().sum()

In [ ]:
df_final['PropensityScore'] = df_final['PropensityScore'].fillna(0) # replacing null values in propensity with zero

In [ ]:
df_final.drop('LoanNumber', axis = 1, inplace=True)

In [ ]:
df_final = df_final[['loan_num_srvcr','AsOfDt','PropensityScore','cluster']]

In [ ]:
df_final.columns = ['loan_num_srvcr', 'AsOfDt','propensity', 'cluster']

In [ ]:
df_final.head()

In [ ]:
df_final['cluster'].value_counts()

**Threshold**

In [ ]:
dictionary = pd.read_csv('port_thresholds_rerun.csv').get('threshold')

In [ ]:
#calculating if it's a high or low within that cluster
df_final['high_low'] = df_final.apply(lambda x: 'High' if x.propensity >= dictionary[x.cluster] else 'Low', axis = 1)

In [ ]:
df_final['cluster'] = df_final['cluster'].map("{:02}".format)

In [ ]:
df_final.head()

In [ ]:
df_final['high_low'].value_counts()

In [ ]:
df_final[df_final['propensity'].isnull()].groupby(['cluster', 'high_low']).count().reset_index()

In [ ]:
df_final['segment'] = 'Port_CRMdl_'+df_final['cluster'].astype('str')+'_'+df_final['high_low']

In [ ]:
df_final.head()

In [ ]:
df_final['segment'].value_counts()

In [ ]:
df_final['RecordCounter'] = df.index+1

df_final.head()

In [ ]:
df_final = df_final[['RecordCounter','AsOfDt','loan_num_srvcr','cluster','propensity','segment']]

df_final.head()

In [ ]:
df_final.columns = ['RecordCounter','Date','LoanNumSrvcr','Cluster','Prop Score','Final Cluster']
df_final.head()

In [ ]:
df_final['Final Cluster'].value_counts()

In [ ]:
df_final['Final Cluster'].value_counts(normalize=True)*100

In [ ]:
# 20% Random loans are made as Control Group
control_group = df_final[~df_final['Prop Score'].isnull()].sample(frac=0.2) # excluding NaN propensity from control group
control_group.shape

In [ ]:
control_group.head()

In [ ]:
df_final['LoanNumSrvcr'] = df_final['LoanNumSrvcr'].astype('int64')

In [ ]:
control_group['LoanNumSrvcr'] = control_group['LoanNumSrvcr'].astype('int64')

In [ ]:
df_final['LoanNumSrvcr'].dtype

In [ ]:
control_group['LoanNumSrvcr'].dtype

In [ ]:
df_final['Final Cluster'] = np.where(df_final['LoanNumSrvcr'].isin(control_group['LoanNumSrvcr']),
                                     'Port_Control_Group',
                                     df_final['Final Cluster'])

In [ ]:
df_final['Final Cluster'].value_counts()

In [ ]:
df_final['Final Cluster'].value_counts(normalize=True)*100

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv('DTC_SBR_PortfolioLoanSegment.csv',index=False)

In [ ]:
output_path = r'/mnt/jobs/J189031000_DTCModels/' + os.sep

In [ ]:
output_path

In [ ]:
'{}DTC_SBR_PortfolioLoanSegment.csv'.format(output_path)

In [ ]:
df_final.to_csv('{}DTC_SBR_PortfolioLoanSegment.csv'.format(output_path),index=False)